# Example for loading a SWMF-GM 3D plot file

In [1]:
import spacepy
import spacepy.pybats
import spacepy.pybats.bats

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)
import plotly

import numpy as np
import scipy.interpolate as spint
import scipy.spatial.qhull as qhull
import itertools

from kamodo import Kamodo, kamodofy
import time

This unreleased version of SpacePy is not supported by the SpacePy team.
Qin-Denton/OMNI2 data not found in current format. This module has limited functionality.
Run spacepy.toolbox.update(QDomni=True) to download data


In [2]:
#b Load the data file
mhd = spacepy.pybats.IdlFile('../../../data/3d__ful_1_t00955900_n4632237.out')

In [3]:
# Various things you can explore in this data
#print("\n=> mhd \n",mhd)
print("\n=> mhd.meta \n",mhd.meta)
print("\n=> mhd.keys() \n",mhd.keys())
print("\n=> mhd.attrs \n",mhd.attrs)
#print("\n=> mhd.values() \n",mhd.values())


=> mhd.meta 
 {'file': '../../../data/3d__ful_1_t00955900_n4632237.out', 'format': None, 'endian': 'little', 'iter': 4632237, 'runtime': 345540.0, 'ndim': 3, 'nparam': 10, 'nvar': 15, 'header': 'R R R Mp/cc km/s km/s km/s nT nT nT nPa nT nT nT J/m3 uA/m2 uA/m2 uA/m2_ful33                                                                                                                                                                                                                                                                                                                                                                                                                                       ', 'g': 1.6666699647903442, 'c': 5995.7998046875, 'th': 0.3241580128669739, 'p1': 3.0, 'p2': 1.0, 'p3': 1.0, 'NX': 6.0, 'NY': 6.0, 'NZ': 6.0, 'R': 2.5, 'strtime': '0095h59m00.000s'}

=> mhd.keys() 
 dict_keys(['grid', 'x', 'y', 'z', 'rho', 'ux', 'uy', 'uz', 'bx', 'by', 'bz', 'p', 'b1x', 'b1y', 'b1z', 'e

In [4]:
# Pull grid and value to plot into arrays
mhdvalues = np.array(mhd['p'], dtype=np.float32)
mhdpoints = np.empty([np.size(mhdvalues), 3], dtype=np.float32)
mhdpoints[:,0] = np.array(mhd['x'])
mhdpoints[:,1] = np.array(mhd['y'])
mhdpoints[:,2] = np.array(mhd['z'])

In [5]:
# Setup grid to interpolate to
nX = 141
nY = 81
zpt = 0.

xint = np.linspace(-50, 20, nX)
yint = np.linspace(-20, 20, nY)
zint = zpt
xx, yy, zz = np.meshgrid(xint,yint,zint, indexing = 'xy')

# Use a mask to reduce the size of the arrays to only values that could impace the new data extraction.
qpts = mhdpoints[(mhdpoints[:,0] > (np.amin(xint)-1.1)) & 
                 (mhdpoints[:,0] < (np.amax(xint)+1.1)) &
                 (mhdpoints[:,1] > (np.amin(yint)-1.1)) & 
                 (mhdpoints[:,1] < (np.amax(yint)+1.1)) & 
                 (mhdpoints[:,2] > (np.amin(zint)-1.1)) & 
                 (mhdpoints[:,2] < (np.amax(zint)+1.1))]
qval = mhdvalues[(mhdpoints[:,0] > (np.amin(xint)-1.1)) & 
                 (mhdpoints[:,0] < (np.amax(xint)+1.1)) &
                 (mhdpoints[:,1] > (np.amin(yint)-1.1)) & 
                 (mhdpoints[:,1] < (np.amax(yint)+1.1)) & 
                 (mhdpoints[:,2] > (np.amin(zint)-1.1)) & 
                 (mhdpoints[:,2] < (np.amax(zint)+1.1))]

newxyz = np.ndarray(shape=(np.size(np.reshape(xx,-1)),3), dtype=np.float32)
newxyz[:,0] = np.reshape(xx,-1)
newxyz[:,1] = np.reshape(yy,-1)
newxyz[:,2] = np.reshape(zz,-1)

dd = np.absolute(np.concatenate(( np.array(mhd['x']), np.array([999.], dtype=np.float32) )) - 
                 np.concatenate(( np.array([999.], dtype=np.float32), np.array(mhd['x']) )) )
gridMinDx = np.amin(dd[dd[:]>0.])

@kamodofy(units = 'nPa')
def g_XY(
    xint = np.linspace(-50, 20, nX), 
    yint = np.linspace(-20, 20, nY)):

    # Get values from weights already computed, and reshape 3D to 2D
    result = Zinterpolate(qval, vtx, wts)
    result2=np.reshape(result,(nY,nX))

    return result2

kamodo = Kamodo(g_XY = g_XY)

In [6]:
# Create functions to optimize execution of pieces that make up griddata interpolation
d = 3
def Zinterp_weights(xyz, uvw):
    tri = qhull.Delaunay(qpts)
    simplex = tri.find_simplex(uvw)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uvw - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def Zinterpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

In [7]:
# Time creation of weights (most time consuming part)
tic = time.perf_counter()

vtx, wts = Zinterp_weights(qpts, newxyz)

toc = time.perf_counter()
print(f"Delta time is {toc - tic:0.4f} seconds")

Delta time is 19.8000 seconds


In [8]:
# Time execution of interpolation and create plot.
tic = time.perf_counter()

txt1="Model: BATSRUS,  Run: weili_jiang_031012_4,  minimum dx=" + str(gridMinDx)
fig = kamodo.plot(g_XY = dict())
fig.update_xaxes(title_text="",scaleanchor='y')
fig.update_yaxes(title_text="$Y_{RE}$")
fig.update_layout(
    title_text="$Z_{RE}=0 \; slice$",
    annotations=[
        dict(text="$X_{RE}$", x=0.5, y=-0.10, showarrow=False, xref="paper", yref="paper", font=dict(size=14)),
        dict(text=txt1,
             x=0, y=0, ax=0, ay=0, xanchor="left", xshift=-80, yshift=-45, xref="paper", yref="paper",
             font=dict(size=16, family="sans serif", color="#000000")
        )
    ],
    hoverlabel = dict(font_size=12)
)
iplot(fig)

toc = time.perf_counter()
print(f"Processing time is {toc - tic:0.4f} seconds")

Processing time is 0.9035 seconds


In [9]:
#Check to see if faster interpolation gives same values as full griddata call.
np.allclose(Zinterpolate(qval, vtx, wts), spint.griddata(qpts, qval, newxyz))

True

Will need to adjust min/max plot values.  
Existing values include values inside 2.5, which can be extreme.  
Excluding values inside of 2.9 Re gives more reasonable values.

In [10]:
r1 = Zinterpolate(qval, vtx, wts)
print(np.amin(r1),np.amax(r1))

0.009664899669587612 1.2732750177383423


In [11]:
r = np.ndarray(shape=(len(np.reshape(xx,-1))), dtype=np.float32)
r = np.sqrt(np.square(newxyz[:,0]) + np.square(newxyz[:,1]) + np.square(newxyz[:,2]))
r2 = r1[(r[:] > 2.9)]
print(np.amin(r2),np.amax(r2))

0.017356449738144875 1.2732750177383423


In [12]:
fig = kamodo.plot(g_XY = dict())

In [13]:
fig.layout

Layout({
    'autosize': False,
    'height': 400,
    'margin': {'b': 32, 'pad': 0, 'r': 30, 't': 40},
    'template': '...',
    'title': {'text': '$\\operatorname{g_{XY}}{\\left(xint,yint \\right)} [nPa] = \\lambda{\\left(xint,yint \\right)}$'},
    'width': 700,
    'xaxis': {'title': {'text': '$xint$'}},
    'yaxis': {'title': {'text': '$yint$'}}
})

In [14]:
fig.data

(Contour({
     'x': array([-50. , -49.5, -49. , -48.5, -48. , -47.5, -47. , -46.5, -46. , -45.5,
                 -45. , -44.5, -44. , -43.5, -43. , -42.5, -42. , -41.5, -41. , -40.5,
                 -40. , -39.5, -39. , -38.5, -38. , -37.5, -37. , -36.5, -36. , -35.5,
                 -35. , -34.5, -34. , -33.5, -33. , -32.5, -32. , -31.5, -31. , -30.5,
                 -30. , -29.5, -29. , -28.5, -28. , -27.5, -27. , -26.5, -26. , -25.5,
                 -25. , -24.5, -24. , -23.5, -23. , -22.5, -22. , -21.5, -21. , -20.5,
                 -20. , -19.5, -19. , -18.5, -18. , -17.5, -17. , -16.5, -16. , -15.5,
                 -15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5, -11. , -10.5,
                 -10. ,  -9.5,  -9. ,  -8.5,  -8. ,  -7.5,  -7. ,  -6.5,  -6. ,  -5.5,
                  -5. ,  -4.5,  -4. ,  -3.5,  -3. ,  -2.5,  -2. ,  -1.5,  -1. ,  -0.5,
                   0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,   4.5,
                   5. ,   5.5,  